In [18]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np
import re
import matplotlib.pyplot as plt

import pandas as pd
from tqdm.auto import tqdm
from lxml.html import fromstring
from itertools import cycle

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random

from time import sleep

from skimage import io

from concurrent.futures import ThreadPoolExecutor

import json
import pickle

In [6]:
def get_elite_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    proxies = []
    soup = BeautifulSoup(response.text, 'lxml', parse_only=SoupStrainer('table', {'id': 'proxylisttable'}))
    for entry in soup('tr')[1:]:
        try:
            ip, port, ptype = [entry('td')[x].text for x in [0,1,4]]
            if 'elite' in ptype:
                proxies.append('http://'+':'.join([ip, port]))
        except:
            None
    return proxies

proxies_list = get_elite_proxies()
proxy_pool = cycle(proxies_list)
ua = UserAgent()

def get_soup_simple(url, strain_name=None, strain_attrs=None):
    #headers = {'User-Agent': ua.random}
    #proxy_val = next(proxy_pool)
    #proxies = {"http": proxy_val, "https": proxy_val}
    #proxies = {"http": None, "https": None}
    
    successful = False
    while not successful:
        try:
            response = requests.get(url)#, headers=headers, proxies=proxies)
            if strain_name is None and strain_attrs is None:
                soup = BeautifulSoup(response.text, 'lxml')
            elif strain_attrs is None:
                strainer = SoupStrainer(strain_name)
                soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
            else:
                strainer = SoupStrainer(strain_name, strain_attrs)
                soup = BeautifulSoup(response.text, 'lxml', parse_only=strainer)
            successful = True
        except BaseException as e:
            continue
    return soup

In [7]:
base_url = 'https://arachno.piwigo.com/'
families_url = base_url + 'index?/category/familles'

main_page = get_soup_simple(families_url, 'li', {'class':'selected liOpen'})

In [10]:
df = pd.read_json('data/spiderbase_0.json')

In [59]:
def group_images(img_content, species):
    groups = {'general': [], 'epigyne': [], 'palpe': [], 'vulve': [], 'other': []}
    for img in img_content:
        name = img['name'].lower()
        if 'epigyne' in name:
            groups['epigyne'].append(img)
        elif 'palpe' in name:
            groups['palpe'].append(img)
        elif 'vulve' in name:
            groups['vulve'].append(img)
        elif name == species.lower():
            groups['general'].append(img)
        else:
            groups['other'].append(img)
    return groups

def get_image_metainfo(args):
    url, species_name = args
    img_content = []
    sleep(1)
    try:
        img_data = get_soup_simple(url, 'ul', {'class':'thumbnails', 'id':"thumbnails"})('span', 'wrap2')
        for dt in img_data:
            sleep(1)
            img_site_link = base_url+dt.a['href']
            name = dt.img.attrs['alt']
            credits = dt.img.attrs['title']
            main_image = get_soup_simple(img_site_link, 'img', {'id': 'theMainImage'}).img
            link = base_url+main_image.attrs['src']
            image = io.imread(link)
            img_content.append({'name': name, 'image': image , 'credits': credits})

        groups = group_images(img_content, species_name)
        #pickle.dump(groups, open(f'data/image_data/{species_name.replace(" ","-").lower()}.pkl','wb'))
        print('fix this')
        return True, args, ''
    except Exception as e:
        return False, args, e
    
def get_image_metainfo_quick(args):
    url, species_name = args
    img_content = []
    sleep(1)
    try:
        img_data = get_soup_simple(url, 'ul', {'class':'thumbnails', 'id':"thumbnails"})('span', 'wrap2')
        for dt in img_data:
            sleep(0.1)
            name = dt.img.attrs['alt']
            credits = dt.img.attrs['title']
            link = base_url+dt.img['src']
            image = io.imread(link)
            img_content.append({'name': name, 'image': image , 'credits': credits})

        groups = group_images(img_content, species_name)
        with open(f'data/image_data/{species_name.replace(" ","-").lower()}.pkl','wb') as out_file:
            pickle.dump(groups, out_file)
        return True, args, ''agelenoides
    except Exception as e:
        return False, args, e

In [61]:
arg_list= df[['link', 'species']].values.astype(str)

results = []
for arg in tqdm(arg_list):
    results.append(get_image_metainfo_quick(arg))

In [62]:
results

[(True,
  array(['https://arachno.piwigo.com/index?/category/1768-agelena_agelenoides',
         'Agelena agelenoides'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/67-agelena_labyrinthica',
         'Agelena labyrinthica'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/68-allagelena_gracilens',
         'Allagelena gracilens'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/1430-aterigena_ligurica',
         'Aterigena ligurica'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/1593-aterigena_soriculata',
         'Aterigena soriculata'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/591-coelotes_atropos',
         'Coelotes atropos'], dtype='<U80'),
  ''),
 (True,
  array(['https://arachno.piwigo.com/index?/category/1375-coelotes_osellai',
         'Coelotes osellai'], dtype='<U80'),
  ''),
 (True,
  ar

In [63]:
results = np.array(results)

In [64]:
results

array([[True,
        array(['https://arachno.piwigo.com/index?/category/1768-agelena_agelenoides',
       'Agelena agelenoides'], dtype='<U80'),
        ''],
       [True,
        array(['https://arachno.piwigo.com/index?/category/67-agelena_labyrinthica',
       'Agelena labyrinthica'], dtype='<U80'),
        ''],
       [True,
        array(['https://arachno.piwigo.com/index?/category/68-allagelena_gracilens',
       'Allagelena gracilens'], dtype='<U80'),
        ''],
       ...,
       [True,
        array(['https://arachno.piwigo.com/index?/category/1302-zodarion_timidum',
       'Zodarion timidum'], dtype='<U80'),
        ''],
       [True,
        array(['https://arachno.piwigo.com/index?/category/608-zoropsis_media',
       'Zoropsis media'], dtype='<U80'),
        ''],
       [True,
        array(['https://arachno.piwigo.com/index?/category/535-zoropsis_spinimana',
       'Zoropsis spinimana'], dtype='<U80'),
        '']], dtype=object)

In [72]:
mask = results[:,0].astype(bool) 

In [73]:
mask

array([ True,  True,  True, ...,  True,  True,  True])

In [75]:
results[~mask]

array([], shape=(0, 3), dtype=object)